#  Singlet scalar singlet (charged) fermion dark matter model

Sample of notebook for specific model

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import pandas as pd
import numpy as np
import os, sys, inspect
import commands3
commands=commands3
from hep import *


## Check one point

In [5]:
a=hep(MODEL='SimplifiedDMSSSFDM')

`a-object` is an object with many attributes and methods. Use the tab to explore them. Some of them are
* a.Series: [pandas](http://pandas.pydata.org/) Series object with the "relevant" variables 
* a.LHA: Input LesHouces file as [pyslha](https://pypi.python.org/pypi/pyslha/) object
* a.runSPheno() -> a.LHA_out: return LHA output files as [pyslha](https://pypi.python.org/pypi/pyslha/) object
* a.runmicromegas() -> a.runSPheno() -> Updated the `a-object`  with micrOMEGAS "relevant" output

In [31]:
v=a.vev
lambda_1=0.26
lambda_SH=0.1
MS=50**2
MF=100
Yse=1.
Ymu=0
Ytau=0
devnull=commands.getoutput('rm -f SPheno.spc.%s' %a.MODEL)
a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
a.LHA.blocks['SPHENOINPUT'].entries[520]='0.             # Write effective Higgs couplings (HiggsBounds blocks): put 0 to use file with MadGraph! '
a.LHA.blocks['MINPAR'][1]='%0.8E       # lambda1'  %lambda_1
a.LHA.blocks['MINPAR'][2]='%0.8E       # lambdaSH'  %lambda_SH
a.LHA.blocks['MINPAR'][4]='%0.8E       # MS'  %MS
a.LHA.blocks['MINPAR'][5]='%0.8E       # MF' %MF
a.LHA.blocks['YSIN'][1]='%0.8E       # Ys(1)' %Yse
a.LHA.blocks['YSIN'][2]='%0.8E       # Ys(1)' %Ymu
a.LHA.blocks['YSIN'][3]='%0.8E       # Ys(3)' %Ytau
moc=a.runmicromegas(Direct_Detection=True, ddcmd='CalcOmega_with_DDetection_MOv4.3')
print('Omega h^2, SI proton, neutron ='),a.Series.Omega_h2,a.Series.proton_SI,a.Series.neutron_SI

Omega h^2, SI proton, neutron =


(None, 0.0030699999999999998, 2.6400000000000001e-08, 2.7129999999999999e-08)

In [ ]:
import time
st=time.time()
a=hep(MODEL='SimplifiedDMSSSFDM')
v=a.vev
Omega_h2_delta=0.0022
CL=3
Omega_h2=0.1197
Omega_h2_exp=[Omega_h2-CL*Omega_h2_delta,Omega_h2,Omega_h2+CL*Omega_h2_delta]
lambda_1=0.26
lambda_SH=0
a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
a.LHA.blocks['SPHENOINPUT'].entries[520]='0.             # Write effective Higgs couplings (HiggsBounds blocks): put 0 to use file with MadGraph! '

df=pd.DataFrame()
a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
dfmin=100 #40
dfmax=200 #1E4
npoints=1000
df_masses=np.logspace(np.log10(dfmin),np.log10(dfmax),npoints) #np.array([200]) 1E-4
DEBUG=False
for i in range(1):
    j=0
    for MF in df_masses:
        print(j)
        j=j+1
        rm1=10**np.random.uniform(np.log10(1E-2),np.log10(3))
        r=rm1+1.
        M_S=MF/r
        MS2=M_S**2-a.vev**2*lambda_SH
        Yse_range=np.logspace(np.log10(np.pi),np.log10(1E-3),200)
        Omega_h2_old=1E32
        k=0
        for Yse in Yse_range:
            print(k,end='\r')
            k=k+1
            devnull=commands.getoutput('rm -f SPheno.spc.%s' %a.MODEL)
            Ymu=0. #10**np.random.uniform( log10(1E-3),np.log10(np.pi) )
            Ytau=0. #10
            a.LHA.blocks['MINPAR'][1]='%0.8E       # lambda1'  %lambda_1
            a.LHA.blocks['MINPAR'][2]='%0.8E       # lambdaSH'  %lambda_SH
            a.LHA.blocks['MINPAR'][4]='%0.8E       # MS2'  %MS2
            a.LHA.blocks['MINPAR'][5]='%0.8E       # MF' %MF
            a.LHA.blocks['YSIN'][1]='%0.8E       # Ys(1)' %Yse
            a.LHA.blocks['YSIN'][2]='%0.8E       # Ys(2)' %Ymu
            a.LHA.blocks['YSIN'][3]='%0.8E       # Ys(3)' %Ytau        
            a.LHA.blocks['MINPAR'][5]='%0.8E       #MF' %MF  # -> Fill a.Series.MDF
            if hasattr(a, 'Series'):
                Omega_h2_old=a.Series.Omega_h2
            
            a.runmicromegas(Direct_Detection=True,ddcmd='CalcOmega_with_DDetection_MOv4.3')
            if DEBUG:
                print( MF,M_S,Yse,a.Series.Omega_h2,np.abs(Omega_h2_old-a.Series.Omega_h2) )
            if a.Series.Omega_h2>Omega_h2_exp[0] and a.Series.Omega_h2<Omega_h2_exp[2]:
                
                a.Series=a.Series.append(block_to_series(a.LHA_out_with_comments.blocks['MASS']))
                kk=a.branchings(a.LHA_out.decays,min_pdg=25)
                a.Series=a.Series.append(a.Br_names)
                df=df.append(a.Series,ignore_index=True)
                df.to_excel('tmp.xlsx',index=False)
                #Escape small lambdas
                if np.abs(Omega_h2_old-Omega_h2)<1E-3:
                    break
            if a.Series.Omega_h2>Omega_h2_exp[2]: 
                break
                
        print('')        
        if DEBUG:
            print(  df.shape,MF,M_S,rm1,a.Series.Omega_h2  )
        if np.where(df_masses==MF)[0][0]%100==0: #find the index of the array entry
            print( np.where(df_masses==MF)[0][0] )            
            
print(st-time.time())

0
17
0
1
38
2
15
3
199
4
24
5
199
6
199
7
199
8
5
9
199
10
12
11
25
12
199
13
7
14
0
15
199
16
0
17
26
18
199
19
199
20
45
21
81
22
199
23
35
24
22
25
18
26
0
27
24
28
5
29
6
30
2
31
199
32
199
33
7
34
22
35
0
36
199
37
21
38
199
39
199
40
199
41
7
42
19
43
199
44
0
45
199
46
16
47
6
48
199
49
199
50
199
51
199
52
47
53
25
54
199
55
24
56
199
57
21
58
22
59
2
60
199
61
199
62
199
63
14
64
19
65
58
66
26
67
199
68
0
69
11
70
199
71
199
72
199
73
199
74
199
75
199
76
12
77
13
78
14
79
199
80
26
81
15
82
20
83
199
84
21
85
25
86
17
87
199
88
199
89
3
90
199
91
13
92
23
93
199
94
22
95
9
96
0
97
49
98
199
99
18
100
199
100
101
29
102
40
103
6
104
28
105
27
106
22
107
49
108
199
109
0
110
78
111
199
112
21
113
5
114
11
115
20
116
199
117
68
118
199
119
20
120
199
121
24
122
27
123
13
124
199
125
12
126
199
127
199
128
5
129
21
130
199
131
1
132
28
133
30
134
199
135
9
136
22
137
199
138
199
139
199
140
14
141
47
142
0
143
199
144
13
145
24
146
15
147
19
148
18
149
199
150
199
151
16
152
29


In [ ]:
pd.read_excel('tmp.xlsx').shape

In [36]:
a.runmicromegas(Direct_Detection=True,ddcmd='CalcOmega_with_DDetection_MOv4.3')

'Masses of odd sector Particles:\n~Ss  : MSs   =   108.8 || ~Fre : MFre  =   112.8 || \nXf=2.64e+01 Omega h^2=2.27e-02\n\n# Channels which contribute to 1/(omega) more than 1%.\n# Relative contributions in % are displayed\n   12% ~Fre ~fre ->u1 U1 \n   12% ~Fre ~fre ->u2 U2 \n   11% ~Fre ~fre ->A A \n   10% ~Fre ~fre ->e2 E2 \n   10% ~Fre ~fre ->e3 E3 \n    9% ~Fre ~fre ->e1 E1 \n    5% ~Fre ~fre ->A Z \n    4% ~Fre ~fre ->d1 D1 \n    4% ~Fre ~fre ->d2 D2 \n    4% ~Fre ~fre ->d3 D3 \n    3% ~Ss ~Fre ->A e1 \n    3% ~Fre ~fre ->nu1 Nu1 \n    3% ~Fre ~fre ->nu2 Nu2 \n    3% ~Fre ~fre ->nu3 Nu3 \n    2% ~Fre ~fre ->Wp Wm \n    1% ~Ss ~Fre ->Z e1 \n\n==== Calculation of CDM-nucleons amplitudes  =====\n         TREE LEVEL\nCDM-nucleon micrOMEGAs amplitudes:\nproton:  SI  0.000E+00  SD  0.000E+00\nneutron: SI  0.000E+00  SD  0.000E+00\n         BOX DIAGRAMS\nCDM-nucleon micrOMEGAs amplitudes:\nproton:  SI  0.000E+00  SD  0.000E+00\nneutron: SI  0.000E+00  SD  0.000E+00\nCDM-nucleon cross sec

In [13]:
a.Series.Omega_h2

0.0030699999999999998

In [11]:
a.runmicromegas?

In [4]:
v=a.vev
devnull=commands.getoutput('rm -f SPheno.spc.%s' %a.MODEL)
a.LHA.blocks['MODSEL'].entries[1]='0               #  1/0: High/low scale input '
a.LHA.blocks['SPHENOINPUT'].entries[6]=  '0         # One-loop decays'
a.LHA.blocks['SPHENOINPUT'].entries[55]= '0          # Calculate one loop masses'
a.LHA.blocks['SPHENOINPUT'].entries[52]= '1          # Write spectrum in case of tachyonic states'
a.LHA.blocks['SPHENOINPUT'].entries[520]='0.         # (HiggsBounds blocks): put 0 to use file with MadGraph! '
a.LHA.blocks['MINPAR'].entries[1]='1.0000000E-01  # Lambda1Input'

In [5]:
spc=a.runSPheno()

In [ ]:
a.LHA_out.blocks['MASS'].entries[25]

In [ ]:
a.LHA_out_with_comments.blocks['MASS'].entries[25]

[![Home](http://www.incredimail.com/images/nav%20bar/home-icon.png)](./) 
[Jupyter home](./draft.pdf)